In [3]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
sx = qt.sigmax();sy = qt.sigmay(); sz = qt.sigmaz(); s0 = qt.qeye(2); sm = qt.sigmam(); sp = qt.sigmap()

In [46]:
# Define the Hamiltonian [[E13, t23,t01,0],[t23,E12, 0, t01],[t01,0,E03,t23],[0,t01,t23,E02]]
def hamiltonian(E13,E12,E03,E02,t23,t01):
    return np.array([[E13, t23,t01,0],[t23,E12, 0, t01],[t01,0,E03,t23],[0,t01,t23,E02]])


# convert Hamiltonian to qutip object and add contribution from resonance gate
def hamiltonian_qutip(E13,E12,E03,E02,t23,t01, lever_arms_res, Vr):
    E13 = E13 + Vr*(lever_arms_res[1]+lever_arms_res[3])
    E12 = E12 + Vr*(lever_arms_res[1]+lever_arms_res[2])
    E03 = E03 + Vr*(lever_arms_res[0]+lever_arms_res[3])
    E02 = E02 + Vr*(lever_arms_res[0]+lever_arms_res[2])
    return qt.Qobj(hamiltonian(E13,E12,E03,E02,t23,t01))


def compute_q_capacitance(Hvr, lever_arms_res,  Vr = [-1e-3,1e-3]):
    Es = []
    for V in Vr:
        Es.append(Hvr(V).eigenstates()[1][0])
    dP_states = np.array(Es[1])**2 - np.array(Es[0])**2
    dP1 = dP_states[2] + dP_states[3]
    dP3 = dP_states[0] + dP_states[2]
    dV = Vr[1]-Vr[0]
    return (lever_arms_res[1]-lever_arms_res[0])*dP1/dV+(
        lever_arms_res[3]-lever_arms_res[2])*dP3/dV



lever_arms_res = np.array([np.exp(-r) for r in range(4)])
#Energies of each state E_ij = mu_i + mu_j + U_{ij}
E02 = 0
E12 = 1
E13 = 2
E03 = 2

# Tunneling rates
t = 1

Hvr = lambda Vr: hamiltonian_qutip(E13,E12,E03,E02,t,t,lever_arms_res, Vr)

# Compute the capacitance
C = compute_q_capacitance(Hvr, lever_arms_res, Vr = [-1e-4,1e-4])

print(C)

[0.07965259+0.j]
